In [ ]:
import numpy as np
from pathlib import Path

def read_timestep(path: str | Path) -> np.ndarray:
    reader = iter(open(path, "r"))
    outs = []
    while True:
        l = next(reader, None)
        if l is None:
            break
        if "ITEM: TIMESTEP" in l:
            t = int(next(reader).strip())
            outs.append(t)
    return np.array(outs)

In [ ]:
import re
def read_timestep_2(path: str | Path):
    with open(path, "r") as f:
        l = f.read()
        t = re.findall(r"ITEM: TIMESTEP\s+(\d+)", l)
        return np.array(t, dtype = int)

print(read_timestep_2("/Users/supercgor/Library/CloudStorage/OneDrive-個人/Documents/Data/simu/disordered-pbc/t120-traj1.pos"))

%timeit read_timestep_2("/Users/supercgor/Library/CloudStorage/OneDrive-個人/Documents/Data/simu/disordered-pbc/t120-traj1.pos")

In [ ]:
import re
from pathlib import Path
from io import StringIO
from shppy.io import read_lammps_dump_text
# from ase.io.lammpsrun import read_lammps_dump_text
from ase.io.extxyz import read_xyz
path = "/Users/supercgor/Library/CloudStorage/OneDrive-個人/Documents/Data/simu/disordered-pbc/t120-traj1.xyz"

%timeit list(read_xyz(open(path, "r"), index = slice(0,-1, 2)))
# print(len(read_lammps_dump_text(path, slice(0,-1, 10))))
# print(read_lammps_dump_text(path, index = -1))

path = "/Users/supercgor/Library/CloudStorage/OneDrive-個人/Documents/Data/simu/disordered-pbc/t120-traj1.pos"

# read_lammps_dump_text(path, index = slice(0,-1, 100), sort_by_id = False)
%timeit read_lammps_dump_text(path, index = slice(0,-1, 2), sort_by_id = False)


In [ ]:
import numpy as np
import numba as nb
from typing import Sequence

cell =  np.array([0.5, 0.5, 0.5])
rs = np.random.rand(10000, 3)

def pbc_map(rs: np.ndarray, cell: np.ndarray | None = None, pbc: Sequence[bool] | bool = False, align_center = False):
    if cell is None or not pbc:
        return rs
    
    if isinstance(pbc, bool):
        pbc = [pbc] * rs.shape[-1]
        
    cell = np.diag(cell) if cell.ndim == 1 else cell
    
    norm_rs = np.linalg.inv(cell.T) @ rs.T # (3, 3) @ (3, N) -> (3, N)
    for i in range(rs.shape[-1]):
        if pbc[i]:
            if align_center:
                norm_rs[i] -= np.rint(norm_rs[i])
            else:
                norm_rs[i] -= np.floor(norm_rs[i])
            
    return (cell.T @ norm_rs).T

out = pbc_map(rs, cell, pbc = True, align_center = True)

%timeit pbc_map(rs, cell, pbc = True, align_center = True)

pbc_map_2 = nb.njit(pbc_map)

pbc_map_2(rs, cell, pbc = True, align_center = True)

%timeit pbc_map_2(rs, cell, pbc = True, align_center = True)
